In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
df=pd.read_csv('train.csv')
df.shape

(404290, 6)

In [6]:
new_df=df.sample(30000)
new_df

,id,qid1,qid2,question1,question2,is_duplicate
52835,52835,93494,93495,How much discount does a TCS employee gets on ...,What is it like to be a GET in Tata Motors?,0
299940,299940,109229,40719,Are there any celebs on Quora?,Who are the celebrities who are actively parti...,1
265634,265634,382647,66176,Will someone tell me what happened to Kejriwal...,What would happen if Modi died now?,0
197328,197328,298258,267085,What is a suitable solar panel installation pr...,What is a suitable solar panel installation pr...,0
401814,401814,280529,535256,What is the acceptable TDS level of drinking w...,How do I calculate water TDS?,0
...,...,...,...,...,...,...
370938,370938,501532,501533,What are the best and worst TV commercials see...,What are some of the best Indian TV commercials?,1
298028,298028,420458,22507,How do you communicate with animals?,How do animals communicate?,0
240966,240966,352896,352897,Internet Advertising: Are there equivalents of...,What is the best thing you have done for your ...,0
164109,164109,223100,99131,"In Pokemon Ranger and The Temple of The Sea, W...",Why is Manaphy childish in Pokémon Ranger and ...,1


In [4]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [5]:
new_df.duplicated().sum()

0

In [7]:
ques_df=new_df[['question1','question2']]

In [17]:
ques_df.dropna(inplace=True)

C:\Users\RESHAB\AppData\Local\Temp\ipykernel_2124\2343159209.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ques_df.dropna(inplace=True)


In [18]:
from sklearn.feature_extraction.text import CountVectorizer
questions = list(ques_df['question1'])+list(ques_df['question2'])

In [26]:
cv = CountVectorizer(max_features=2000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [27]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(29999, 4000)

In [28]:
temp_df['is_duplicate']=new_df['is_duplicate']

In [29]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2,random_state=1)
 

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7405

In [32]:
pip install xgboost

     ---------------------------------------- 70.9/70.9 MB 2.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [33]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)

0.7311666666666666

In [36]:
q1 = 'Where is the capital of India?'
q2 = 'What is the current capital of Pakistan?'
q3 = 'Which city serves as the capital of India?'
q4 = 'What is the business capital of India?'
rf.predict(q1)

ValueError: could not convert string to float: 'Where is the capital of India?'